In [1]:
import numpy as np
import pandas as pd

In [68]:
# Считываем данные из текстового файла
data = pd.read_csv('input.csv', skipinitialspace=True)
data

,name,type,n_tokens,out_edges
0,P1,0,1,T1
1,P2,0,1,T1
2,T1,1,0,P3
3,P3,0,0,T2 T9
4,T9,1,0,P5
5,P5,0,0,T3
6,T3,1,0,P3
7,T2,1,0,P4
8,P4,0,0,T4 T5
9,T4,1,0,P6


In [69]:
# Делим на состояния и переходы
places = data[data['type'] == 0].reset_index()[['name', 'n_tokens', 'out_edges']]
places

,name,n_tokens,out_edges
0,P1,1,T1
1,P2,1,T1
2,P3,0,T2 T9
3,P5,0,T3
4,P4,0,T4 T5
5,P6,0,T6
6,P7,0,T7
7,P8,0,T8
8,P9,0,-


In [70]:
transitions = data[data['type'] == 1].reset_index()[['name', 'out_edges']]
transitions 

,name,out_edges
0,T1,P3
1,T9,P5
2,T3,P3
3,T2,P4
4,T4,P6
5,T5,P7
6,T6,P8
7,T7,P4
8,T8,P1 P2
9,T9,NaN


In [71]:
# Поскольку граф двудольный, будем хранить две матрицы,
# ссылающиеся друг на друга

# Начальная разметка
state0 = np.zeros(places.shape[0])
# Ребра от состояний к переходам
adj_matr_p = np.zeros([places.shape[0], transitions.shape[0]])
for idx_curr, [name, n_tokens, out_edges_str] in places.iterrows():
    # Пробрасываем ребра
    out_edges = out_edges_str.split()
    for name_i in out_edges:
        idx_to = transitions[transitions['name'] == name_i].index[0]
        adj_matr_p[idx_curr, idx_to] = 1
    # Прописываем количество меток
    state0[idx_curr] = n_tokens
    
        

# Ребра от переходов к состояниям
adj_matr_t = np.zeros([transitions.shape[0], places.shape[0]])
for idx_curr, [name, out_edges_str] in transitions.iterrows():
    # Пробрасываем ребра
    out_edges = out_edges_str.split()
    for name_i in out_edges:
        idx_to = places[places['name'] == name_i].index[0]
        adj_matr_t[idx_curr, idx_to] = 1


IndexError: index 0 is out of bounds for axis 0 with size 0

In [55]:
# А лучше два листа межности

# Начальная разметка
state0 = np.zeros(places.shape[0])
# Ребра от состояний к переходам
adj_list_p = {}
for idx_curr, [name, n_tokens, out_edges_str] in places.iterrows():
    # Сначала ребра в лист
    adj_list_p[idx_curr] = []
    out_edges = out_edges_str.split()
    for name_i in out_edges:
        idx_to = transitions[transitions['name'] == name_i].index[0]
        adj_list_p[idx_curr].append(idx_to)
    # Для удобства лист ребер в массив
    adj_list_p[idx_curr] = np.array(adj_list_p[idx_curr]) 
    # Прописываем количество меток
    state0[idx_curr] = n_tokens

# Ребра от переходов к состояниям
adj_list_t = {}
for idx_curr, [name, out_edges_str] in transitions.iterrows():
    adj_list_t[idx_curr] = []
    out_edges = out_edges_str.split()
    for name_i in out_edges:
        idx_to = places[places['name'] == name_i].index[0]
        adj_list_t[idx_curr].append(idx_to)
    adj_list_t[idx_curr] = np.array(adj_list_t[idx_curr]) 


In [67]:
# Неплохо проверить, что у нас нет тупиковых вершин
# Если вершина тупиковая, её вектор нулевой
for key in adj_list_p:
    if adj_list_p[key].size == 0:
        print('WARNING: Тупиковая позиция', places['name'][key])        

for key in adj_list_t:
    if adj_list_t[key].size == 0:
        print('WARNING: Тупиковый переход', transitions['name'][key])

In [66]:
# Теперь непосредственно строим граф достижимости

'P2'